In [1]:
'''
@author Adrian Spork
@author Tatjana Melina Walter
'''

'\n@author Adrian Spork\n@author Tatjana Melina Walter\n'

In [2]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import getpass
import xarray as xr
import rasterio as rio
import os
import pandas as pd
import numpy as np
import shutil
from time import sleep
import stat
import io
from rasterio.enums import Resampling
import netCDF4 as nc
from datetime import datetime
from zipfile import ZipFile

In [23]:
def downloadingData(aoi, collectionDate, plName, prLevel, clouds, username, password):
    '''
    Downloads the Sentinel2-Data with the given parameters

    Parameters:
        aoi (str): The type and the coordinates of the area of interest
        collectionDate datetime 64 [ns]): The date of the data
        plName (str): The name of the platform
        prLevel (str): The name of the process
        clouds: (tuple of ints) Min and max of cloudcoverpercentage
    '''

    api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

    '''Choosing the data with bounding box (footprint), date, platformname, processinglevel and cloudcoverpercentage'''
    products = api.query(aoi, date = collectionDate, platformname = plName, processinglevel = prLevel, cloudcoverpercentage = clouds)

    '''Filters the products and sort by cloudcoverpercentage'''
    products_gdf = api.to_geodataframe(products)
    products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending = [True])

    '''Downloads the choosen files from Scihub'''
    products_gdf_sorted.to_csv(os.path.join(directory, 'w'))
    api.download_all(products, directory, max_attempts = 10, checksum = True)

In [4]:
def unzipping(filename):
    '''
    Unzips the file with the given filename

    Parameter:
        filename(str): Name of the .zip file
    '''
    with ZipFile(os.path.join(directory, filename), 'r') as zipObj:
        zipObj.extractall(directory)

In [5]:
def unzip(directory):
    '''
    Unzips and deletes the .zip in the given directory

    Parameters:
        directory (str): Pathlike string to the directory
    '''

    for filename in os.listdir(directory):
        if filename.endswith(".zip"):
            unzipping(filename)
            delete(os.path.join(directory, filename))
            continue
        else:
            continue

In [6]:
def delete(path):
    '''
    Deletes the file/directory with the given path

    Parameters:
        path (str): Path to the file/directory
    '''

    if os.path.exists(path):
        os.remove(path)
        print("File deleted: " + path)
    else:
        print("The file does not exist")

In [7]:
def extractBands(filename, resolution):
    '''
    Extracts bandpaths from the given .SAFE file

    Parameters:
        filename (str): Sentinel .SAFE file
        resolution (int): The resolution the datacube should have

    Returns:
        bandPaths (str[]): An array of the paths for the red and nir band
    '''

    lTwoA = os.listdir(os.path.join(directory, filename, "GRANULE"))

    if resolution == 10:
        bandName = os.listdir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[4]))
        bandPaths = [pathRed, pathNIR]

    elif resolution == 20:
        bandName = os.listdir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[9]))
        bandPaths = [pathRed, pathNIR]

    elif resolution == 60:
        bandName = os.listdir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[4]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[11]))
        bandPaths = [pathRed, pathNIR]

    elif resolution == 100:
        bandName = os.listdir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[9]))
        bandPaths = [pathRed, pathNIR]

    else:
        print("No such resolution")
        return -1

    return bandPaths

In [8]:
def loadBand (bandpath, date, tile, resolution, clouds, plName, prLevel):
    '''
    Opens and reads the red and nir band, saves them as NetCDF file

    Parameters:
        bandPaths (str[]): Array with the paths to the red and nir band
        date (datetime 64 [ns]): The collection date ("2020-12-31")
        tile (str): Bounding box of coordinates defined by Sentinel
        resolution (int): The resolution of the dataset
        clouds (tuple of ints): Min and max of cloudcoverpercentage
        plName (str): The name of the platform
        prLevel (str): The level of the process

    Returns:
        dataset (xArray dataset): The result dataset as xArray dataset
    '''

    b4 = rio.open(bandpath[0])
    b8 = rio.open(bandpath[1])
    red = b4.read()
    nir = b8.read()

    if resolution == 10:
        res = 1830 * 3 * 2
    elif resolution == 20:
        res = 1830 * 3
    elif resolution == 60:
        res = 1830
    elif resolution == 100:
        res = 1098
    else:
        print("No such resolution")
        return -1

    j = res - 1
    i = 0
    lat = [0] * res
    lon = [0] * res
    while j >= 0:
        lon[i] = b4.bounds.left + i * resolution
        lat[i] = b4.bounds.bottom + j * resolution
        i = i + 1
        j = j - 1

    time = pd.date_range(date, periods = 1)

    if resolution == 100:
        upscale_factor = (1/5)
        nir = b8.read(
                out_shape = (
                    b8.count,
                    int(b8.height * upscale_factor),
                    int(b8.width * upscale_factor)
                ),
                resampling = Resampling.bilinear
        )
        transform = b8.transform * b8.transform.scale(
            (b8.width / nir.shape[-1]),
            (b8.height / nir.shape[-2])
        )
        red = b4.read(
            out_shape = (
                b4.count,
                int(b4.height * upscale_factor),
                int(b4.width * upscale_factor)
            ),
            resampling = Resampling.bilinear
        )

        transform = b4.transform * b4.transform.scale(
            (b4.width / red.shape[-1]),
            (b4.height / red.shape[-2])
        )

    dataset = xr.Dataset(
        {
            "red": (["time","lat", "lon"], red),
            "nir": (["time","lat", "lon"], nir)
        },
        coords = dict(
            time = time,
            lat = (["lat"], lat),
            lon = (["lon"], lon),
        ),
        attrs = dict(
            platform = plName,
            processingLevel = prLevel,
            cloudcover = clouds,
            source = "https://scihub.copernicus.eu/dhus",
            resolution = str(resolution) + " x " + str(resolution) + " m"
        ),
    )

    dataset.to_netcdf(directory + "datacube_" + str(date) + "_" + str(tile) + "_R" + str(resolution) + ".nc", 'w', format = 'NETCDF4')
    b4.close()
    b8.close()
    return dataset

In [9]:
def getDate(filename):
    '''
    Extracts the Date out of the Sentinelfilename

    Parameters:
        filename (str): Name of the file

    Returns:
        (str): Date of the File ("2020-12-31")
    '''

    return filename[11:15] + "-" + filename[15:17] + "-" + filename[17:19]

In [10]:
def getTile(filename):
    '''
    Extracts the UTM-tile of the Sentinelfilename

    Parameters:
        filename (str): Name of the file

    Returns:
        (str): UTM-tile of the File ("31UMC")
    '''
    return filename[38:44]

In [11]:
def on_rm_error(func, path, exc_info):
    '''
    Unlinks a read-only file
    '''

    os.chmod(path, stat.S_IWRITE)
    os.unlink(path)

In [12]:
def buildCube(directory, resolution, clouds, plName, prLevel):
    '''
    Builds a datacube in the given directory with coords, time as dimensions and the bands as datavariables

    Parameters:
        directory (str): Pathlike string to the directory
        resolution (int): The resolution of the dataset
        clouds (tuple of ints): Min and max of cloudcoverpercentage
        plName (str): The name of the platform
        prLevel (str): The level of the process
    '''

    for filename in os.listdir(directory):
        if filename.endswith(".SAFE"):
            bandPath = extractBands(os.path.join(directory, filename), resolution)
            band = loadBand(bandPath, getDate(filename), getTile(filename), resolution, clouds, plName, prLevel)
            shutil.rmtree(os.path.join(directory, filename), onerror = on_rm_error)
            print(" ")
            continue
        else:
            continue

In [13]:
def merge_Sentinel(directory):
    '''
    Merges datacubes by coordinates and time

    Parameters:
        directory (str): Pathlike string where Data ist stored
    '''

    start = datetime.now()
    count1 = 0
    files = os.listdir(directory)

    if len(files) == 0:
        print("Directory empty")
        return
    elif len(files) == 1:
        print("Only one file in directory")
        return
    else:
        print('Start merging')
        for file1 in files:
            if count1 == len(files):
                return
            for file2 in files:
                count2 = 0
                if file1.endswith(".nc"):
                    file1Date = file1[9:19]
                    file1Tile = file1[20:26]
                    file1Res = file1[27:31]
                    file2Date = file2[9:19]
                    file2Tile = file2[20:26]
                    file2Res = file2[27:31]

                    if file1Date == file2Date and file1Tile == file2Tile and file1Res == file2Res:
                        continue
                    elif file1Date == file2Date and file1Tile == "T32ULC" and file2Tile == "T32UMC" and file1Res == file2Res:
                        fileLeft = xr.open_dataset(os.path.join(directory, file1))
                        fileRight = xr.open_dataset(os.path.join(directory, file2))
                        merge_coords(fileLeft, fileRight, file1[0:20] + "Merged" + file1[26:31])
                        fileLeft.close()
                        fileRight.close()
                        delete(os.path.join(directory, file1))
                        delete(os.path.join(directory, file2))
                        continue
                else:
                    print("Error: Wrong file in directory")
                    continue

        files = os.listdir(directory)
        while len(os.listdir(directory)) > 1:
            files = os.listdir(directory)
            file1 = xr.open_dataset(os.path.join(directory, files[0]))
            file2 = xr.open_dataset(os.path.join(directory, files[1]))
            merge_time(file1, file2, files[0][0:31])
            file1.close()
            file2.close()
            delete(os.path.join(directory, files[1]))
            continue

    end = datetime.now()
    diff = end - start
    print('All cubes merged for ' + str(diff.seconds) + 's')
    return

In [14]:
def timeframe(ds, start, end):
    '''
    Slices Datacube down to given timeframe

    Parameters:
        ds (ds): Sourcedataset
        start (str): Start of the timeframe eg '2018-07-13'
        end (str): End of the timeframe eg '2018-08-23'

    Returns:
        ds_selected (ds): Dataset sliced to timeframe
    '''

    if start > end:
        print("start and end of the timeframe are not compatible!")
    else:
        ds_selected = ds.sel(time = slice(start, end))
        return ds_selected  

In [15]:
def safe_datacube(ds, name):
    '''
    Saves the Datacube as NetCDF (.nc)

    Parameters:
        ds (ds): Sourcedataset
        name (str): Name eg '2017', '2015_2019'
    '''

    print("Start saving")
    if type(name) != str:
        name = str(name)
    ds.to_netcdf(directory + name + ".nc")
    print("Done saving")

In [16]:
def merge_coords(ds_left, ds_right, name):
    '''
    Merges two datasets by coordinates

    Parameters:
        ds_left (xArray dataset): Dataset to be merged
        ds_right (xArray dataset): Dataset to be merged
        name (str): Name of the new dataset
    '''

    ds_selected = slice_lon(ds_left, ds_left.lon[0], ds_right.lon[0])
    ds_merge = [ds_selected, ds_right]
    merged = xr.combine_by_coords(ds_merge)
    safe_datacube(merged, name)

In [17]:
def merge_time(ds1, ds2, name):
    '''
    Merges two datasets by time

    Parameters:
        ds1 (xArray dataset): Dataset to be merged
        ds2 (xArray dataset): Dataset to be merged
        name (str): Name of the new dataset
    '''

    res = xr.combine_by_coords([ds1, ds2])
    ds1.close()
    ds2.close()
    safe_datacube(res, name)

In [18]:
def slice_lat(ds, lat_left, lat_right):
    '''
    Slices a given dataset to given latitude bounds

    Parameters:
        ds (xArray Dataset): Dataset to be sliced
        lat_left (float): Left latitude bound
        lat_right (float): Right latitude bound

    Returns:
        ds (xArray Dataset): Sliced dataset
    '''

    ds_selected = ds.sel(lat = slice(lat_left, lat_right))
    return ds_selected

In [19]:
def slice_lon(ds, lon_left, lon_right):
    '''
    Slices a given dataset to given longitude bounds

    Parameters:
        ds (xArray Dataset): Dataset to be sliced
        lon_left (flaot): Left longitude bound
        lon_right (float): Right longitude bound

    Returns:
        ds (xArray Dataset): Sliced dataset
    '''

    ds_selected = ds.sel(lon = slice(lon_left, lon_right))
    return ds_selected

In [20]:
def slice_coords(ds, lon_left, lon_right, lat_left, lat_right):
    '''
    Slices a dataset to a given slice

    Parameters:
        ds (xArray Dataset): Dataset to be sliced
        lon_left (float): Left bound for longitude
        lon_right (float): Right bound for longitude
        lat_left (float): Left bound for latitude
        lat_right (float): Right bound for latitude

    Returns:
        ds (xArray Dataset): Sliced dataset
    '''

    ds_selcted = slice_lon(ds, lon_left, lon_right)
    return slice_lat(ds_selected, lat_left, lat_right)

In [25]:
def main(resolution, directory, collectionDate, aoi, clouds, username, password):
    '''
    Downloads, unzips, collects and merges Sentinel2 Satelliteimages to a single netCDF4 datacube

    Parameters:
        resolution (int): Resolution of the satelite image
        directory (str): Pathlike string to the workdirectory
        collectionDate (tuple of datetime 64[ns]): Start and end of the timeframe
        aoi (POLYGON): Area of interest
        clouds (tuple of ints): Min and max of cloudcoverpercentage
        username (str): Uername for the Copernicus Open Acess Hub
        password (str): Password for the Copernicus Open Acess Hub
    '''

    plName = 'Sentinel-2'
    prLevel = 'Level-2A'
    downloadingData (aoi, collectionDate, plName, prLevel, clouds, username, password)
    delete(os.path.join(directory,'w'))
    unzip(directory)
    buildCube(directory, resolution, clouds, plName, prLevel)
    merge_Sentinel(directory)

Execution

In [27]:
directory = 'D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/'
resolution = 100    #10, 20, 60, 100 possible

'''Parameters for the download'''
aoi = 'POLYGON((7.52834379254901 52.01238155392252,7.71417925515199 52.01183230436206,7.705255583805303 51.9153349236737,7.521204845259327 51.90983021961716,7.52834379254901 52.01238155392252,7.52834379254901 52.01238155392252))'
collectionDate = ('20200601', '20200630')
clouds = (0, 30)
username = getpass.getpass("user: ")
password = getpass.getpass("password: ")

main(resolution, directory, collectionDate, aoi, clouds, username, password)

Downloading:   0%|                                                                         | 0.00/1.16G [00:00<?, ?B/s]
Downloading:   0%|▏                                                               | 3.15M/1.16G [00:00<02:13, 8.66MB/s]
Downloading:   0%|▏                                                               | 4.19M/1.16G [00:00<02:13, 8.66MB/s]
Downloading:   1%|▎                                                               | 6.29M/1.16G [00:00<02:37, 7.35MB/s]
Downloading:   1%|▍                                                               | 7.34M/1.16G [00:00<02:34, 7.47MB/s]
Downloading:   1%|▌                                                               | 9.44M/1.16G [00:01<02:36, 7.36MB/s]
Downloading:   1%|▋                                                               | 12.6M/1.16G [00:01<02:17, 8.39MB/s]
Downloading:   1%|▊                                                               | 14.7M/1.16G [00:01<02:09, 8.85MB/s]
Downloading:   1%|▍                     

Downloading:  20%|████████████▉                                                    | 232M/1.16G [00:28<02:30, 6.19MB/s]
Downloading:  20%|█████████████                                                    | 234M/1.16G [00:29<02:24, 6.43MB/s]
Downloading:  20%|█████████████▏                                                   | 235M/1.16G [00:29<02:23, 6.48MB/s]
Downloading:  20%|█████████████▏                                                   | 236M/1.16G [00:29<02:22, 6.53MB/s]
Downloading:  20%|█████████████▏                                                   | 237M/1.16G [00:29<02:17, 6.73MB/s]
Downloading:  20%|█████████████▎                                                   | 238M/1.16G [00:29<02:15, 6.84MB/s]
Downloading:  21%|█████████████▎                                                   | 239M/1.16G [00:30<02:13, 6.91MB/s]
Downloading:  21%|█████████████▍                                                   | 241M/1.16G [00:30<02:10, 7.05MB/s]
Downloading:  21%|█████████████▌        

Downloading:  32%|████████████████████▋                                            | 369M/1.16G [00:50<02:00, 6.58MB/s]
Downloading:  32%|████████████████████▋                                            | 370M/1.16G [00:50<01:59, 6.61MB/s]
Downloading:  32%|████████████████████▋                                            | 371M/1.16G [00:50<01:57, 6.75MB/s]
Downloading:  32%|████████████████████▊                                            | 372M/1.16G [00:50<01:58, 6.69MB/s]
Downloading:  32%|████████████████████▊                                            | 373M/1.16G [00:51<01:59, 6.62MB/s]
Downloading:  32%|████████████████████▉                                            | 374M/1.16G [00:51<01:58, 6.66MB/s]
Downloading:  32%|████████████████████▉                                            | 375M/1.16G [00:51<01:55, 6.81MB/s]
Downloading:  32%|█████████████████████                                            | 376M/1.16G [00:51<01:58, 6.66MB/s]
Downloading:  32%|█████████████████████ 

Downloading:  34%|██████████████████████▏                                          | 416M/1.22G [01:12<01:51, 7.16MB/s]
Downloading:  44%|████████████████████████████▎                                    | 508M/1.16G [01:12<02:11, 4.98MB/s]
Downloading:  44%|████████████████████████████▍                                    | 509M/1.16G [01:12<02:30, 4.34MB/s]
Downloading:  34%|██████████████████████▍                                          | 419M/1.22G [01:12<02:09, 6.13MB/s]
Downloading:  44%|████████████████████████████▍                                    | 510M/1.16G [01:12<02:22, 4.58MB/s]
Downloading:  44%|████████████████████████████▌                                    | 511M/1.16G [01:13<02:16, 4.79MB/s]
Downloading:  44%|████████████████████████████▌                                    | 512M/1.16G [01:13<02:12, 4.92MB/s]
Downloading:  35%|██████████████████████▋                                          | 424M/1.22G [01:13<01:50, 7.15MB/s]
Downloading:  44%|██████████████████████

Downloading:  54%|███████████████████████████████████                              | 627M/1.16G [01:32<01:35, 5.60MB/s]
Downloading:  46%|█████████████████████████████▋                                   | 556M/1.22G [01:32<01:29, 7.38MB/s]
Downloading:  54%|███████████████████████████████████                              | 628M/1.16G [01:32<01:41, 5.27MB/s]
Downloading:  54%|███████████████████████████████████▏                             | 629M/1.16G [01:32<01:40, 5.31MB/s]
Downloading:  54%|███████████████████████████████████▏                             | 630M/1.16G [01:33<01:39, 5.36MB/s]
Downloading:  46%|█████████████████████████████▉                                   | 560M/1.22G [01:33<01:30, 7.28MB/s]
Downloading:  54%|███████████████████████████████████▎                             | 631M/1.16G [01:33<01:38, 5.38MB/s]
Downloading:  54%|███████████████████████████████████▎                             | 632M/1.16G [01:33<01:37, 5.45MB/s]
Downloading:  54%|██████████████████████

...

Downloading:  94%|████████████████████████████████████████████████████████████▏   | 1.09G/1.16G [03:36<00:10, 6.56MB/s]
Downloading:  94%|████████████████████████████████████████████████████████████▎   | 1.09G/1.16G [03:37<00:10, 6.50MB/s]
Downloading:  94%|████████████████████████████████████████████████████████████▎   | 1.10G/1.16G [03:37<00:10, 6.59MB/s]
Downloading:  94%|████████████████████████████████████████████████████████████▍   | 1.10G/1.16G [03:37<00:10, 6.54MB/s]
Downloading:  94%|████████████████████████████████████████████████████████████▍   | 1.10G/1.16G [03:37<00:09, 6.61MB/s]
Downloading:  95%|████████████████████████████████████████████████████████████▌   | 1.10G/1.16G [03:37<00:09, 6.55MB/s]
Downloading:  95%|████████████████████████████████████████████████████████████▌   | 1.10G/1.16G [03:37<00:09, 6.55MB/s]
Downloading:  95%|████████████████████████████████████████████████████████████▌   | 1.10G/1.16G [03:38<00:09, 6.63MB/s]
Downloading:  95%|██████████████████████

Downloading:  82%|█████████████████████████████████████████████████████▏           | 988M/1.21G [02:44<00:16, 13.0MB/s]
Downloading:  82%|█████████████████████████████████████████████████████▎           | 990M/1.21G [02:44<00:16, 13.0MB/s]
Downloading:  82%|█████████████████████████████████████████████████████▍           | 992M/1.21G [02:44<00:16, 13.1MB/s]
Downloading:  82%|█████████████████████████████████████████████████████▌           | 994M/1.21G [02:44<00:16, 13.0MB/s]
Downloading:  83%|█████████████████████████████████████████████████████▋           | 996M/1.21G [02:44<00:16, 13.1MB/s]
Downloading:  83%|█████████████████████████████████████████████████████▊           | 998M/1.21G [02:45<00:15, 13.2MB/s]
Downloading:  83%|█████████████████████████████████████████████████████           | 1.00G/1.21G [02:45<00:15, 13.2MB/s]
Downloading:  83%|█████████████████████████████████████████████████████▏          | 1.00G/1.21G [02:45<00:15, 13.1MB/s]
Downloading:  83%|██████████████████████

File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/w
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/S2A_MSIL2A_20200613T103031_N0214_R108_T32UMC_20200613T111252.zip
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/S2A_MSIL2A_20200623T103031_N0214_R108_T32ULC_20200623T142851.zip
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/S2A_MSIL2A_20200623T103031_N0214_R108_T32UMC_20200623T142851.zip
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/S2A_MSIL2A_20200626T104031_N0214_R008_T32ULC_20200626T135731.zip
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/S2A_MSIL2A_20200626T104031_N0214_R008_T32UMC_20200626T135731.zip
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel

<ipython-input-15-74b61d9fef0a>:13: SerializationWarning: saving variable red with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf(directory + name + ".nc")
<ipython-input-15-74b61d9fef0a>:13: SerializationWarning: saving variable nir with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf(directory + name + ".nc")



File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/datacube_2020-06-13_T32UMC_R100.nc
Start saving
Done saving
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/datacube_2020-06-23_Merged_R100.nc
Start saving
Done saving
File deleted: D:/Tatjana/Documents/Studium/Semester 5 - Abgaben/Geosoftware 2/Code/Sentinel_Data/datacube_2020-06-26_Merged_R100.nc
All cubes merged for 1s
